In [99]:
import pandas as pd
import requests
from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

url = "https://raw.githubusercontent.com/MyungKyuYi/AI-class/main/titanic.csv"

response = requests.get(url)

data = StringIO(response.text)
df = pd.read_csv(data)

df.columns
# print(f"생존자 수: {num_survived}")

df.head()
print(df.columns)
df = df.drop(columns=["Name","Cabin","Embarked","Ticket"])
print(df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare'],
      dtype='object')


In [101]:
df.isnull().sum() #결측치 파악

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

In [103]:
#Age의 결측치가 177개 발견되었으므로 결측치를 평균값으로 채워넣는다.
df["Age"] = df["Age"].fillna(df["Age"].mean()) 

#sex 란 데이터를 숫자로 변환
le = LabelEncoder()
df["Sex"] = le.fit_transform(df["Sex"]) #male -> 1로 female -> 0으로 바꾸기

In [109]:
#생존여부가 예측해야될 값이므로 X에는 survived 제외 Y에만 survived 넣기
X = df.drop(columns=["Survived"])
Y = df["Survived"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

scaler = StandardScaler() #정규화 (아직 잘 모르겠음 왜 정규화 하는지, 후에 더 알아보도록 한다.)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#여기서도 문제가 있었음 Y데이터 들이 오류가 나서 GPT물어보니 Numpy 배열로 변환해야한다함 왜 인지는 모르겠음, 이것도 나중에 알아볼 예정
Y_train = Y_train.values.ravel()
Y_test = Y_test.values.ravel()

models = {
    "KNN": KNeighborsClassifier(n_neighbors=5), 
    "RF": RandomForestClassifier(n_estimators=100, random_state=42),
    "DT": DecisionTreeClassifier(random_state=42), 
    "RL": LogisticRegression(max_iter=1000), 
    "SVM": SVC()
}

for name, model in models.items():
    if name in ["KNN", "SVM"]: #거리기반 알고리즘이라서 따로 실행해줘야함.
        
        """ 여기서 계속 문제가 있었음 원인을 몰라 계속 찾아본 결과 결측치가 687개나 되는 cabin은 드랍시켰지만 
        Embarked에 Nan이라는 데이터가 들어있는걸 보지못하고 그대로 진행하여 fit 학습 과정에서 문자열때문에 오류가 났던것을 발견함. 
        Embarked를 드롭하면서 문제 해결"""
        
        model.fit(X_train_scaled, Y_train)   
        Y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, Y_train) #나머지들 실행
        Y_pred = model.predict(X_test)

    # 정확도 계산
    accuracy = accuracy_score(Y_test, Y_pred)
    print(f"{name} 정확도: {accuracy:.4f}")

KNN 정확도: 0.7985
RF 정확도: 0.8097
DT 정확도: 0.7575
RL 정확도: 0.8097
SVM 정확도: 0.8284
